In [1]:
from dotenv import load_dotenv
import os
import openai
import time
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
import warnings

load_dotenv()
warnings.filterwarnings("ignore")


In [2]:
DATA_PATH = "data/training_data.jsonl"

def train_model():
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    # Upload file
    print("Uploading file to OpenAI...")
    upload_response = client.files.create(
        file=open(DATA_PATH, "rb"),
        purpose="fine-tune"
    )
    uploaded_file_id = upload_response.id
    print("File uploaded. File ID:", uploaded_file_id)

    # Create fine-tune job
    print("Creating fine-tune job...")
    fine_tune_response = client.fine_tuning.jobs.create(
        training_file=uploaded_file_id,
        model="gpt-3.5-turbo-0125",
    )
    fine_tune_id = fine_tune_response.id
    print("Fine-tune job created. Response:", fine_tune_response)

    # Wait for completion
    print("Waiting for fine-tune job to complete... This can take a while.")
    status = None
    while status not in ["succeeded", "failed"]:
        time.sleep(30)  # wait before checking
        job_status = client.fine_tuning.jobs.retrieve(fine_tune_id)
        status = job_status.status
        print(f"Current status: {status}")
    
    print("Fine-tune job finished with status:", status)
    
    # Save the fine-tuned model name
    if status == "succeeded":
        fine_tuned_model = job_status.fine_tuned_model
        print("Fine-tuned model name:", fine_tuned_model)
        with open("fine_tuned_model_name.txt", "w") as f:
            f.write(fine_tuned_model)


In [3]:
prompt_template = """
You are a specialized model that outputs JSON schemas for the AdvancedFormNode.
User requirement: {user_requirement}
Return the valid JSON schema. 
"""

def get_llm_chain():
    # Read fine-tuned model name
    with open("fine_tuned_model_name.txt", "r") as f:
        fine_tuned_model = f.read().strip()
        
    # Setup ChatOpenAI with updated parameters
    llm = ChatOpenAI(
        model_name=fine_tuned_model,
        temperature=0.0,
        api_key=os.getenv("OPENAI_API_KEY")
    )
    
    # Create prompt template using langchain-core
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["user_requirement"]
    )
    
    # Create chain with updated LLMChain
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=False
    )
    
    return chain

In [ ]:
# Call this function to train the model
train_model()

In [ ]:
chain = get_llm_chain()

In [ ]:
user_requirement = "Give me schema for the Advanced form node which contain a Form and has the following fields: First Name, last name, email, phone number"
result = chain.run(user_requirement=user_requirement)